### 직방 서비스 원룸 데이터 수집
- 복잡한 데이터 수집 방법
- 동이름 > 매물정보(데이터프레임)

- 절차
    - 동이름 > 위도, 경도
    - 위도, 경도 > geohash code(지역범위)
    - geohash > 매물 아이디
    - 매물 아이디 > 매물 정보

In [5]:
import pandas as pd
import requests

In [1]:
# 1. 동이름 > 위도, 경도

In [11]:
query = "마포구 합정동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={query}&serviceType=원룸"
response = requests.get(url)
datas = response.json()["items"][0]
lat, lng = datas["lat"], datas["lng"]
lat, lng

(37.549537658691406, 126.90560913085938)

In [2]:
# 2. 위도, 경도 > geohash

In [13]:
# !pip install geohash2

In [14]:
import geohash2

In [17]:
# precision : 영역 : 값이 커질수록 영역이 작아짐
code = geohash2.encode(lat, lng, precision=5)
code

'wydjx'

In [3]:
# 3. geohash > 매물 아이디

In [20]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
response = requests.get(url)
response

<Response [200]>

In [24]:
datas = response.json()["items"]
datas[:3]

[{'lat': 37.52911584984866, 'lng': 126.89761140063823, 'item_id': 30462357},
 {'lat': 37.529618523515886, 'lng': 126.89757894127727, 'item_id': 30524930},
 {'lat': 37.52911607377812, 'lng': 126.89697440047796, 'item_id': 30629925}]

In [25]:
ids = []
for data in datas:
    ids.append(data["item_id"])
ids[:5]

[30462357, 30524930, 30629925, 30681505, 30494154]

In [26]:
# list comprehention
ids = [data["item_id"] for data in datas]
ids[:3]

[30462357, 30524930, 30629925]

In [31]:
# list comprehention : 간단한 리스트 데이터를 만들때
# 홀수 데이터만 제곱해서 리스트 만들어 출력
result = [data**2 for data in range(10) if data % 2]
result

[1, 9, 25, 49, 81]

In [4]:
# 4. 매물 아이디 > 매물 정보

In [33]:
ids[:3]

[30462357, 30524930, 30629925]

In [34]:
url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain": "zigbang",
    "withCoalition": "true",
    "item_ids": ids,
}
response = requests.post(url, params)
response

<Response [200]>

In [37]:
items = response.json()["items"]
item_df = pd.DataFrame(items)
item_df.tail(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
914,None,30455640,https://ic.zigbang.com/ic/items/30455640/1.jpg,전세,전세,28000,0,33.58,"{'m2': 33.58, 'p': '10.2'}","{'m2': 23.98, 'p': '7.3'}",...,False,True,빌라,[추천],서울시 마포구 중동,None,None,3,2022-02-03T12:00:40+09:00,False
915,None,30687168,https://ic.zigbang.com/ic/items/30687168/1.jpg,월세,월세,5000,60,41.36,"{'m2': 41.36, 'p': '12.5'}","{'m2': 36.36, 'p': '11'}",...,False,True,빌라,[],서울시 마포구 중동,None,None,8,2022-02-18T14:00:36+09:00,False


In [40]:
pd.options.display.max_columns = 30

In [42]:
pd.options.display.max_rows

60

In [45]:
columns = [
    'item_id', 'sales_title', 'deposit', 'rent', 'size_m2',
    'floor', 'building_floor', 'title', 'room_type', 'address', 
    'service_type', 'address1', 'manage_cost', 'reg_date', 'is_new',
]

In [47]:
query

'마포구 합정동'

In [52]:
result_df = item_df[columns]
result_df = result_df[result_df["address"].str.contains(query)]
result_df = result_df.reset_index(drop=True)
result_df.tail(1)

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
90,30723130,전세,33000,0,50.0,4,5,■합정역/망원역세권■구조좋고 넓은2룸■넓은테라스■베란다,04,마포구 합정동,빌라,서울시 마포구 합정동,5,2022-02-21T10:27:19+09:00,True


In [55]:
# 보증금 1억 이하, 월세 100만원 이하
# result_df[(result_df["deposit"] <= 10000) & (result_df["rent"] <= 100)]

In [56]:
def oneroom(address):
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType=원룸"
    response = requests.get(url)
    datas = response.json()["items"][0]
    lat, lng = datas["lat"], datas["lng"]
    code = geohash2.encode(lat, lng, precision=5)
    
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    datas = response.json()["items"]
    ids = [data["item_id"] for data in datas]
    
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "true",
        "item_ids": ids,
    }
    response = requests.post(url, params)
    items = response.json()["items"]
    item_df = pd.DataFrame(items)
    
    columns = [
        'item_id', 'sales_title', 'deposit', 'rent', 'size_m2',
        'floor', 'building_floor', 'title', 'room_type', 'address', 
        'service_type', 'address1', 'manage_cost', 'reg_date', 'is_new',
    ]
    
    result_df = item_df[columns]
    result_df = result_df[result_df["address"].str.contains(address)]
    return result_df.reset_index(drop=True)

In [57]:
oneroom("망원동")

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
0,30483493,전세,23000,0,18.99,6,6,"🦋합정역,신축1.5룸가성비짱,한강공원도보,보증보험OK",02,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-04T15:40:23+09:00,False
1,30608827,전세,23000,0,42.98,6,6,방1개 별도거실 주방 화장실 다용도실 분리형원룸 입니다,04,마포구 망원동,빌라,서울시 마포구 망원동,7,2022-02-12T16:27:27+09:00,False
2,30578522,월세,12000,10,12.21,3,5,💥LH 가능 깨끗한 방💥,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-15T18:54:14+09:00,False
3,30726074,월세,12000,10,13.22,3,5,🎶 [LH 가능!] 🎶 [깔끔한 방!],01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-21T11:45:06+09:00,True
4,30340227,월세,1000,49,23.14,3,5,⭕망원역세권 7평원룸 내부깔끔⭕,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-01-22T14:38:50+09:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,30573911,전세,37000,0,36.17,2,6,마포구청역 3분.전세대출가능.무융자.신축급 컨디션 투룸~!,04,마포구 망원동,빌라,서울시 마포구 망원동,4.3,2022-02-10T13:55:31+09:00,False
90,30606067,전세,37000,0,39.67,2,6,🌷 대출가능! 투룸! 깔끔한 인테리어! 6호선! 🌷,04,마포구 망원동,빌라,서울시 마포구 망원동,5,2022-02-16T18:36:13+09:00,False
91,30713445,월세,5000,30,19.83,3,4,🎢마포구청역5분 신축3년차🎢,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-21T13:46:30+09:00,True
92,30594723,월세,200,55,16.53,1,3,💖1.5층 단기방! 마포구청역 3분!💖,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-14T09:49:15+09:00,False
